<a href="https://colab.research.google.com/github/nomi181472/MS-DS-Practice/blob/main/Deployments/Running_Flask_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/models/'

fashion.hdf5		       sa_encoder.vocab.tokens1
fashion_tpu.hdf5	       sentiment_analysis.hdf5
final_sentiment_analysis.hdf5  sentiment_analysis.hdf51
sa_encoder.vocab.tokens


In [ ]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.2


In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
import threading

app = Flask(__name__)
padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab Notebooks/models/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6000}).start()

Model and Vocabalory loaded.......
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:6000/ (Press CTRL+C to quit)


In [ ]:
import requests
req = requests.get("http://172.28.0.2:6000/")
print(req.status_code)
print(req.text)

172.28.0.2 - - [05/Jun/2020 14:25:01] "GET / HTTP/1.1" 200 -


200
I am alive!


In [ ]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

Still working my way through it but definitely changes your view on investment. Wish it was available on Audible


172.28.0.2 - - [05/Jun/2020 14:25:35] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[0.8370056748390198]],"sentiment ":"positive"}



In [ ]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

I thought this will be good one. But I was wrong


172.28.0.2 - - [05/Jun/2020 14:26:08] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[-0.8676939606666565]],"sentiment ":"Negative"}



In [ ]:
!ps -eaf | grep python

root          18       8  0 13:59 ?        00:00:03 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         126      18  0 14:04 ?        00:00:08 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-4b17b2d1-f76b-46f8-a312-6700eb4ce66e.json
root         745     126  0 14:26 ?        00:00:00 /bin/bash -c ps -eaf | grep python
root         747     745  0 14:26 ?        00:00:00 grep python


In [ ]:
import flask
flask.__version__

'1.1.2'

In [ ]:
!pip install flask-ngrok

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/models/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/Colab Notebooks/models/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app.run()

Model and Vocabalory loaded.......
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f06e7206df84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Jun/2020 14:34:45] "GET / HTTP/1.1" 200 -


I thought this will be good one. But I was wrong


127.0.0.1 - - [05/Jun/2020 14:35:01] "POST /seclassifier HTTP/1.1" 200 -
